In [2]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib qt5

In [3]:
from posarmctools.readdata import *
%load_ext autoreload
%autoreload 2

In [9]:
data_date = "2017_10_12_11_52_43"
data_dir = "/home/pleroy/DATA/PoSAR-MC_PIMA_TEST-3/2017_10_12/" + data_date

In [10]:
594/6

99.0

In [22]:
blocksPerFile = 6
samplesPerRamp = 6000
rampsPerFile = 1500
samplesPerFile = samplesPerRamp * rampsPerFile

# selection in record (0 to 594)
firstFile = 0
nbFiles = 99
firstBlock = firstFile * blocksPerFile
lastFile = firstFile + nbFiles

### Read the data

In [23]:
timeSerie_A = np.zeros( samplesPerFile )
timeSerie_B = np.zeros( samplesPerFile )

adc_A = np.zeros( (nbFiles, samplesPerFile) )
adc_B = np.zeros( (nbFiles, samplesPerFile) )

In [49]:
lastBlock = (nbFiles-1) * blocksPerFile + firstBlock
for k in range( nbFiles ):
    
    # read the data
    nb = str( int( k * blocksPerFile + firstBlock) )
    filename = data_dir + "/record" + nb + ".bin"
    readFile( filename, samplesPerFile, timeSerie_A, timeSerie_B )
    
    print("k = {}, block = {} / {}".format(k, nb, lastBlock ))
    
    adc_A[ k, : ] = timeSerie_A
    adc_B[ k, : ] = timeSerie_B

k = 0, block = 0 / 588
k = 1, block = 6 / 588
k = 2, block = 12 / 588
k = 3, block = 18 / 588
k = 4, block = 24 / 588
k = 5, block = 30 / 588
k = 6, block = 36 / 588
k = 7, block = 42 / 588
k = 8, block = 48 / 588
k = 9, block = 54 / 588
k = 10, block = 60 / 588
k = 11, block = 66 / 588
k = 12, block = 72 / 588
k = 13, block = 78 / 588
k = 14, block = 84 / 588
k = 15, block = 90 / 588
k = 16, block = 96 / 588
k = 17, block = 102 / 588
k = 18, block = 108 / 588
k = 19, block = 114 / 588
k = 20, block = 120 / 588
k = 21, block = 126 / 588
k = 22, block = 132 / 588
k = 23, block = 138 / 588
k = 24, block = 144 / 588
k = 25, block = 150 / 588
k = 26, block = 156 / 588
k = 27, block = 162 / 588
k = 28, block = 168 / 588
k = 29, block = 174 / 588
k = 30, block = 180 / 588
k = 31, block = 186 / 588
k = 32, block = 192 / 588
k = 33, block = 198 / 588
k = 34, block = 204 / 588
k = 35, block = 210 / 588
k = 36, block = 216 / 588
k = 37, block = 222 / 588
k = 38, block = 228 / 588
k = 39, block =

In [50]:
A_reshaped = adc_A.reshape(nbFiles * rampsPerFile, samplesPerRamp)
B_reshaped = adc_B.reshape(nbFiles * rampsPerFile, samplesPerRamp)

In [26]:
plt.figure()
plt.plot( A_reshaped[0, :] )
plt.title("first acquisition\n" + data_date)
plt.grid()

In [51]:
coupling_A = np.average(A_reshaped, 0)
#A_reshaped -= coupling_A

In [48]:
plt.figure()
plt.plot( coupling_A )
plt.title("coupling\n" + data_dir)
plt.grid()

In [29]:
ifft_a = np.fft.ifft( (A_reshaped[::10,0:3000]), axis = 1 )

In [30]:
plt.figure()
vmin = -30
vmax = 0
nbPointsInRange = 2000
plt.plot(  20 * np.log10( np.abs( ifft_a[2500,0:nbPointsInRange] ) ) )
plt.plot( np.ones( nbPointsInRange ) * vmin, "b" )
plt.plot( np.ones( nbPointsInRange ) * vmax, "r" )

In [31]:
plt.matshow( 20 * np.log10( np.abs( ifft_a[:,0:1000] ) ), 
            aspect='auto', vmin=-30, vmax=0, cmap='jet' )
plt.title("Track 1 - record 2017_10_12_11_47_05\nFiles {} to {}".format(firstFile, lastFile) )

In [33]:
plt.matshow( 20 * np.log10( np.abs( ifft_a[:,300:1000] ) ), 
            aspect='auto', cmap='jet' )
plt.title("Track 1 - record 2017_10_12_11_47_05\nFiles {} to {}".format(firstFile, lastFile) )

In [32]:
plt.figure()
plt.plot( np.angle( ifft_a[9500:11500,192] ) )

In [27]:
np.save( data_dir + "/selection_files_{}_{}".format(firstFile, lastFile), A_reshaped )

In [41]:
sys.path.insert(0, "/home/pleroy/DEV/processing/focalization_python")
from posarutils.other.read_data_and_build_rd import *
from posarutils.other.PosarMCParameters import *

In [45]:
params_filename = data_dir + "/" + data_date + "_parameters.xml"
params = PosarMCParameters( params_filename )

shape of the samples matrix = (148500, 6000)


In [52]:
RDc = build_rd_from_data( params, A_reshaped )

shape of the samples matrix = (148500, 6000)


In [53]:
coupling_RD_local = np.average(RDc, 0)
np.save( data_dir + "/coupling_RD", coupling_RD_local )

In [54]:
plt.figure()
plt.plot(np.abs(coupling_RD_local))
plt.grid()